In [102]:
#i followed the following tutorial
#https://towardsdatascience.com/latent-variables-expectation-maximization-algorithm-fb15c4e0f32c
#https://github.com/suvoooo/Machine_Learning/blob/master/ExMax_ALgo/LVM.ipynb

import math
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd


In [103]:
data = np.array([[1, 2],[4, 2], [1,3], [4,3]])
data2 = np.array([[1,4,1,4], [2,2,3,3]])
data2

array([[1, 4, 1, 4],
       [2, 2, 3, 3]])

In [104]:
#### E step for GMM 
### Use multivariate normal for Gaussian distribution 
from scipy.stats import multivariate_normal

def E_step(data, pi, mu, sigma):
  N = data.shape[0] # number of data-points
  K = pi.shape[0] # number of clusters, following notation used before in description
    
  d = mu.shape[1] # dimension of each data point, think of these as attributes
  
  gamma = np.zeros((N, K)) # this is basically responsibility which should be equal to posterior. 

  for nk in range(K):
    gamma[:, nk] = pi[nk] * multivariate_normal.pdf(data, mean=mu[nk], cov=sigma[nk]) 
    # calculate responsibility for each cluster
  gamma = gamma/np.sum(gamma, axis=1, keepdims=True) 
  # use the sum over all the clusters, thus axis=1. Denominator term. 
  # print ("gamma shape: ", gamma.shape)
  return gamma

In [105]:

def M_step(data, gamma):
  N, D = data.shape 
  K = gamma.shape[1] # use the posterior shape calculated in E-step to determine the no. of clusters  
  pi = np.zeros(K)
  mu = np.zeros((K, D))
  sigma = np.zeros((K, D, D)) 

  for ik in range(K):
    n_k = gamma[:, ik].sum() # we use the definintion of N_k 
    pi[ik] = n_k/N # definition of the weights
    elements = np.reshape(gamma[:, ik], (gamma.shape[0], 1)) 
    # get each columns and reshape it (K, 1) form so that later broadcasting is possible. 
    mu[ik,:] = (np.multiply( elements,  data)).sum(axis=0) / n_k  
    sigma_sum = 0.
    for i in range(N):
      var = data[i] - mu[ik]
      sigma_sum = sigma_sum + gamma[i, ik] * np.outer(var, var)# outer product creates the covariance matrix
    sigma[ik,:] = sigma_sum/n_k    
  return pi, mu, sigma

In [106]:

def elbo(data, gamma, pi, mu, sigma):
  N = data.shape[0] # no. of data-points
  K = gamma.shape[1] # no. of clusters
  d = data.shape[1] # dim. of each object

  loss = 0.
  for i in range(N):
      x = data[i]
      for k in range(K):
        pos_dist = gamma[i, k] ## p(z_i=k|x) = gamma_ik
        log_lik = np.log(multivariate_normal.pdf(x, mean=mu[k, :], cov=sigma[k, :, :]) + 1e-20) # log p(x|z)
        log_q = np.log(gamma[i, k] + 1e-20) # log q(z) = log p(z_i=k|x)
        log_pz = np.log(pi[k] + 1e-20)  # log p(z_k =1) =\pi _k
        loss = (loss + np.multiply(pos_dist, log_pz) + np.multiply(pos_dist, log_lik) +  
        np.multiply(pos_dist, -log_q) )
  #print ("check loss: ", loss)

  return loss

In [107]:
def train_loop(data, K, tolerance=1e-3, max_iter=50, restart=10):
  N, d = data.shape
  elbo_best = -np.inf # loss set to the lowest value 
  pi_best = None
  mu_best = None
  sigma_best = None
  gamma_f = None
  for _ in range(restart):
    pi = np.ones(K) / K # if 3 clusters then an array of [.33, .33, .33] # the sum of pi's should be one 
    # that's why normalized  
    #you can change this to match the slides
    mu = np.random.rand(K, d) # no condition on 
    #mu = np.array([2.5, 2.5])
    sigma = np.tile(np.eye(d), (K, 1, 1)) # to initialize sigma we first start with ones only at the diagonals
    #sigma = np.tile([1.7321, 0.57735])
    # the sigmas are postive semi-definite and symmetric  
    last_iter_loss = None
    all_losses = []
    try:

      for i in range(max_iter):
        gamma = E_step(data, pi, mu, sigma)
        pi, mu, sigma = M_step(data, gamma)
        loss = elbo(data, gamma, pi, mu, sigma)
        if loss > elbo_best:
          elbo_best = loss
          pi_best = pi 
          mu_best = mu
          sigma_best = sigma
          gamma_f = gamma
        if last_iter_loss and abs((loss-last_iter_loss)/last_iter_loss) < tolerance: # insignificant improvement
          break 
        last_iter_loss = loss
        all_losses.append(loss)
    except np.linalg.LinAlgError: # avoid the delta function situation 
      pass 

  return elbo_best, pi_best, mu_best, sigma_best, all_losses, gamma_f

In [116]:
best_loss, pi_best, mu_best, sigma_best, ls_lst, final_posterior = train_loop(data, 2)

In [117]:

print ("starting loss, best_loss: ", ls_lst[0], ',',  best_loss)
print ("best pi", pi_best) 
print ("best mu: ", mu_best)

print ("best sigma: ", sigma_best)

starting loss, best_loss:  -10.196246223377141 , -0.8434791419504601
best pi [0.27238556 0.72761444]
best mu:  [[1.00000002 2.43793519]
 [3.06153139 2.52323423]]
best sigma:  [[[ 5.41145113e-08 -1.27739223e-09]
  [-1.27739223e-09  2.46147959e-01]]

 [[ 1.93468250e+00 -4.78980852e-02]
  [-4.78980852e-02  2.49460171e-01]]]
